In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4' # nvidia-smi로 비어있는 gpu 확인하고 여기서 선택할것!

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)


from asset.tokenization_kobert import KoBertTokenizer
from asset import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

#from huggingface_hub import notebook_login

#notebook_login()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [2]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [3]:
model = EncoderDecoderModel.from_pretrained('./dacon-v0/checkpoint-30000')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2
import pandas as pd

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
smodel = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [5]:
# 문장 예시
preds = [
    "이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.",
    "데이콘은 플랫폼입니다."
]

gts = [
    "이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.",
    "데이콘은 국내 최대의 AI 경진대회 플랫폼입니다."
]

In [6]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [7]:
sample_scores = []
for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 512 Embedding Vector로 변환
    pred_embed = smodel.encode(pred)
    gt_embed = smodel.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))

예측 :  이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.
정답 :  이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.
Cosine Similarity Score :  0.83436465
--------------------
예측 :  데이콘은 플랫폼입니다.
정답 :  데이콘은 국내 최대의 AI 경진대회 플랫폼입니다.
Cosine Similarity Score :  0.6281873
--------------------
전체 샘플의 Cosine Similarity Score 평균 :  0.731276


In [8]:
input_list = []


data = pd.read_csv("./bigdata/train.csv")
input_dict = {"utterance":[]}
for _, row in data.iterrows():
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            input_list.append({"question":row[q_col],"answer":row[a_col]})
            


In [9]:
def read_input(path):
    test_question = []
    test = pd.read_csv(path)
    for ut in test['질문']:
        test_question.append(ut)

    return test_question

In [10]:
test_q= read_input('./bigdata/test.csv')

In [11]:
gt_q_embed = []

for gt_q_dic in input_list:
    gt_q_embed.append({"question_embed":smodel.encode(gt_q_dic["question"]),"question":gt_q_dic["question"],"answer":gt_q_dic["answer"]})
    


In [12]:


def translate(text):
    max_sim_dic = {"question_embed":None,"question":None,"answer":None}
    max_sim = -100
    text_embed = smodel.encode(text)
    for gt_dic in gt_q_embed:
        tmp_sim = cosine_similarity(gt_dic["question_embed"], text_embed)
        if tmp_sim > max_sim:
            if "장점" or "단점" in text:
                if ("장점" in text and "단점" in gt_dic["question"]) or ("단점" in text and "장점" in gt_dic["question"]):
                    #print("there was an error")
                    continue
            max_sim = tmp_sim
            max_sim_dic = {"question_embed":gt,"question":gt_dic["question"],"answer":gt_dic["answer"]}
    
    
    if max_sim < 0.9:
        
        embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
        embeddings = {k: v for k, v in embeddings.items()}
        # generate 함수에 하이퍼파라미터 추가
        output_sequences = model.generate(
            **embeddings,
            max_length=65,
            temperature=0.9,
            top_k=1,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
            num_return_sequences=1
        )
    
        tmp = trg_tokenizer.decode(output_sequences[0, 1:-1].cpu())
        return tmp
    
    #print("max_sim is...  " +str(max_sim))
    return max_sim_dic["answer"]

In [13]:
def translate_framework(text):
    pred = ""
    dividers = ["또한,","그리고"]
    special_divider1 = ["와 ","관계는 무엇인가요?"]
    for divider in dividers:
        
        if divider in text:
            text1 = text.split(divider)[0].strip()
            text2 = text.split(divider)[1].strip()
            pred = translate(text1) +" "+divider+" "+ translate(text2)
            return pred
    
    
    
    if special_divider1[1] in text:
        text1 = text.split(special_divider1[0])[0].strip()
        text2 = text.split(special_divider1[0])[1].strip()[:-12]
        pred = translate(text1) +" "+divider+" "+ translate(text2)
        return pred
    
    pred = translate(text)
    return pred
    

In [14]:
preds = []
for i in range(len(test_q)):
    sr_text = test_q[i] # dialect
    tg_text = translate_framework(sr_text) # translate
    preds.append(tg_text)
    if i<30:
        print("source_text : ",sr_text)
        print("result :      ",tg_text)
        print()
        print()

/home/leadawon5/gitfiles/venvs/myvenv/lib/python3.7/site-packages/transformers/generation/utils.py:1260: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"


source_text :  방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?
result :       방청 페인트의 종류로는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트 등이 있습니다. 이러한 페인트들은 각각의 특성과 용도에 맞게 선택하여 사용할 수 있습니다. 광명단페인트는 반사율이 높아서 더 밝은 또한, 원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수축/팽장이 생길 수 있다는 점입니다.


source_text :  도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
result :       도배지에 녹이 묻어나오는 현상은 녹 오염으로 도배지에 붉은색의 녹이 베어나오는 것을 말합니다. 이 현상은 보통 높은 습도나 누수로 인해 발생할 수 있습니다.  고습한 환경에서는 도배지 안쪽의 금속이 누수되어 녹이 생기며, 이로 인해 도배지에 녹오염이 발생할 수 있습니다. 따라서 공간 내 습도를


source_text :  큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?
result :       큐블럭의 주요 단점으로는 일반 벽돌에 비해 비싼 가격과 균열 발생 가능성, 습기로 인한 하자 발생 가능성이 있습니다. 또한, 설치 시 추가적으로 접착재와 전문 기술이 요구되어 비용이 더 들어갈 수 있다는 점도 단점으로 꼽힙니다. 이러한 점들을 감안하여 건축자 및 시공 업체들은 클라이언트에게 건축 재료의 선택과 함께 추가적인 비용과 유지보수에 또한, 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.


source_text :  철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?
result :       철골구조는 건물의 외벽에

In [15]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

(130, 512)

In [16]:
submit = pd.read_csv('./bigdata/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.019636,0.018732,-0.013807,-0.007743,0.077827,0.024059,-0.029655,0.018027,-0.015486,...,-0.006760,-0.034679,-0.014559,-0.026286,-0.020465,-0.006100,0.020319,0.008170,0.022868,0.018702
1,TEST_001,-0.033622,-0.015678,0.015385,-0.005961,0.094560,-0.010433,-0.002295,-0.008866,-0.021159,...,-0.041756,-0.000449,-0.012305,-0.025932,0.002290,0.058562,-0.004484,-0.024566,0.028744,0.022505
2,TEST_002,0.013144,-0.039166,-0.013231,0.009887,0.120874,-0.056810,-0.002036,-0.015133,0.042991,...,-0.020216,-0.027236,0.048518,0.001011,-0.027009,0.006626,-0.001985,-0.029363,-0.017132,0.084847
3,TEST_003,-0.024568,-0.002601,0.003103,0.015793,0.083582,-0.054123,-0.075798,-0.018808,-0.032676,...,-0.032129,-0.016686,0.022896,-0.044920,-0.019058,0.057306,-0.018497,-0.004645,-0.059227,0.069999
4,TEST_004,0.011283,-0.034817,-0.023118,-0.000053,0.092529,-0.017547,0.055174,0.074435,0.024652,...,0.014689,-0.031523,0.037243,-0.008729,-0.035160,0.030983,-0.053108,-0.001524,-0.009388,-0.001211


In [17]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./submit/dacon-v0_submit.csv', index=False)